In [1]:
import json
import time
import subprocess
import torch
import os
import sys
sys.path.append('/workspace/SteerKep/activation-steering')
sys.path.append("/workspace/SteerKep/SteerPoser/src")
sys.path.append("/workspace/SteerKep/steer-data")
sys.path.append("/workspace/SteerKep/RLBench")
torch.cuda.is_available()

True

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from activation_steering import SteeringDataset, MalleableModel, SteeringVector
from arguments import get_config
from user_steered_model import *
steer_cfg = get_config(config_path='/workspace/SteerKep/steer-data/steerconfig.yaml')

In [3]:
model = UserSteeredModel(steer_cfg)

Loading NousResearch/DeepHermes-3-Mistral-24B-Preview into HF cache dir /workspace/.cache/huggingface on device cuda


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

... The target model type is mistral.

In [4]:
mall = model.malleable_model

# Accessing inner layers

In [5]:
from activation_steering import malleable_model as mm
from activation_steering import steering_vector as sv

In [6]:
sv = model.steering_vector

AttributeError: 'UserSteeredModel' object has no attribute 'steering_vector'

In [ ]:
sv

In [24]:
use_chat_template = False
prompt = ["i went to the store and got"]
self = mall
settings = model.settings

In [26]:
mall.device

device(type='cuda', index=0)

In [25]:
if use_chat_template:
    formatted_prompt = self.tokenizer.apply_chat_template(
        [{"role": "user", "content": f"{prompt}"}],
        tokenize=False, add_generation_prompt=True
    )
else:
    formatted_prompt = prompt

input_ids = self.tokenizer(formatted_prompt, return_tensors="pt").to(self.device)

if settings is None:
    settings = {
        "pad_token_id": self.tokenizer.eos_token_id,
        "do_sample": False,
        "max_new_tokens": 50,
        "repetition_penalty": 1.1,
    }

with torch.no_grad():  # Ensure we're not tracking gradients during inference
    output = self.model.generate(**input_ids, **settings)

response = self.tokenizer.decode(output.squeeze()[input_ids['input_ids'].shape[1]:])


In [27]:
formatted_prompt

['i went to the store and got']

In [22]:
input_ids

{'input_ids': tensor([[     1,      1,   1060,   1124,  10460,  35645,   3384,   1124,   1062,
           3263,   1060,   1124,   1474,  35645,   3384,   1124,   3318,   3024,
           1105,   6387,   1317,   1278,   8141,   1321,   4884,   2963, 131072,
           1060,   1124,  10460,  35645,   3384,   1124,   1062,   1503,  19464,
           1060,   1124,   1474,  35645,   3384,   1124,   3318]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}

In [23]:
response

"It seems like you're sharing a sentence fragment. Could you please provide more context or complete the sentence so I can better assist you?<|eot_id|>"

In [13]:
mall.model.generate(["i went to the store and got"])

AttributeError: 'list' object has no attribute 'shape'

In [6]:
layers = mm.get_model_layer_list(mall)

In [11]:
layers[0].layer

MistralDecoderLayer(
  (self_attn): MistralAttention(
    (q_proj): Linear(in_features=5120, out_features=4096, bias=False)
    (k_proj): Linear(in_features=5120, out_features=1024, bias=False)
    (v_proj): Linear(in_features=5120, out_features=1024, bias=False)
    (o_proj): Linear(in_features=4096, out_features=5120, bias=False)
  )
  (mlp): MistralMLP(
    (gate_proj): Linear(in_features=5120, out_features=32768, bias=False)
    (up_proj): Linear(in_features=5120, out_features=32768, bias=False)
    (down_proj): Linear(in_features=32768, out_features=5120, bias=False)
    (act_fn): SiLU()
  )
  (input_layernorm): MistralRMSNorm((5120,), eps=1e-05)
  (post_attention_layernorm): MistralRMSNorm((5120,), eps=1e-05)
)

# Just playing with LLM output

In [49]:
from steered_model import *

In [66]:
model2 = SteeredModel(model.cfg, model.model, model.tokenizer)

Loading NousResearch/DeepHermes-3-Mistral-24B-Preview into HF cache dir /workspace/.cache/huggingface on device cuda


Loading SteeringVector from /workspace/SteerKep/SteerPoser/src/svec/junk-healthy-24b.svec

Loaded directions for layers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 
23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]

Shape of first direction vector: (5120,)

... The target model type is mistral.

Steering...

In [ ]:
model2.update_steer("

In [57]:
unsteer = model2.generate("Went to grocery store and bought", steer=False, debug=False)

In [58]:
steer = model2.generate("Went to grocery store and bought", steer=True, debug=False)

In [61]:
steer

"It seems like you're looking for a list of items you bought at the grocery store. However, you haven't provided any specific items or details about your shopping. Could you please provide more information about what you purchased or what you need help with?"

In [62]:
unsteer

'Went to grocery store and bought some fresh fruits and vegetables. I also bought some fresh fish and some fresh meat. I also bought some fresh herbs and spices. I also bought some fresh nuts and seeds. I also bought some fresh yogurt and some fresh cheese. I also bought some fresh bread and some fresh pasta. I also bought some fresh fruit juice and some fresh vegetable juice. I also bought some fresh honey and some fresh maple syrup. I also bought some fresh almond milk and some fresh coconut milk. I also bought some fresh chia seeds and some fresh flax seeds. I also bought some fresh chia seeds and some fresh flax seeds. I also bought some fresh chia seeds and some fresh flax seeds. I also bought some fresh chia seeds and some fresh flax seeds. I also bought some fresh chia seeds and some fresh flax seeds. I also bought some fresh chia seeds and some fresh flax seeds. I also bought some fresh chia seeds and some fresh flax seeds. I also bought some fresh chia seeds and some fresh fla

In [64]:
steer_cfg.steering_vector_path

'/workspace/SteerKep/SteerPoser/src/svec/junk-healthy-24b'

In [ ]:
SteeringVector.load(cfg.steering_vector_path)

In [65]:
class SteeredModel:
    def __init__(self, cfg, model, tokenizer, verbose=True):
        self.cfg = cfg
        device = "cuda" if torch.cuda.is_available() else "cpu"
        if verbose:
            print(f"Loading {cfg.model_name} into HF cache dir {cfg.cache_dir} on device {device}")
        self.model = model
        self.tokenizer = tokenizer
        #main_device = model.hf_device_map.get(next(iter(self.model.hf_device_map)))
        #main_device = torch.device(f"cuda:{main_device}") if isinstance(main_device, int) else torch.device(main_device)
        self.steering_vector = SteeringVector.load(cfg.steering_vector_path)
        self.malleable_model = MalleableModel(self.model, self.tokenizer)
        self.malleable_model.steer(
            self.steering_vector,
            behavior_layer_ids=cfg.behavior_layer_ids,
            behavior_vector_strength=cfg.behavior_vector_strength,
        )
        #self.malleable_model.device = main_device
        self.settings = {
            "pad_token_id": self.tokenizer.eos_token_id,
            "do_sample": False,
            "max_new_tokens": cfg.max_new_tokens,
            "eos_token_id": self.tokenizer.eos_token_id
        }
        self.model.eval()

    def update_steer(self, steer_vector, strength):
        svpath = os.path.join(cfg.steering_vector_dir, steer_vector)
        self.steering_vector = SteeringVector.load(svpath)
        self.malleable_model.steer(
            self.steering_vector,
            behavior_layer_ids=cfg.behavior_layer_ids,
            behavior_vector_strength=strength,
        )
        
    def generate(self, prompts, steer=True, debug=False):
        # Accept a single string or a list of strings
        if isinstance(prompts, str):
            prompts = [prompts]

        if steer:
            if(debug):
                print("Steering!\n")
            # Steered generation using MalleableModel
            responses = self.malleable_model.respond_batch_sequential(prompts=prompts,settings=self.settings) 
        else:
             # Unsteered generation using regular Hugging Face model
             inputs = self.tokenizer(prompts, return_tensors="pt", padding=True).to(self.model.device)
             outputs = self.model.generate(**inputs, **self.settings)
             responses = self.tokenizer.batch_decode(outputs, skip_special_tokens=True)

        # # If user passed a single prompt, return a single string
        # ret = []
        # for response in responses:
        #     ret.append(response.split('#')[0])

        # print("from the steered model -", ret)
        if(debug):
            print("raw output: ", responses[0], "\n\n")
        responses = [parse_output(txt) for txt in responses]
        responses = responses[0] if len(responses) == 1 else responses
        return responses


In [41]:
model.generate("I went to the grocery store and bought a ")

in steered model I went to the grocery store and bought a variety of fresh fruits and vegetables, lean proteins, whole grains, and healthy snacks. I also picked up some dairy products, nuts, seeds, and a few pantry staples like olive oil, spices, and herbs. I made sure to choose organic and locally sourced produce whenever possible to support local farmers and reduce my environmental impact. I also picked up some fresh herbs and spices to add flavor to my meals without adding extra salt or sugar. I'm excited to cook some healthy and delicious meals with my fresh ingredients!<|eot_id|>
after parsing I went to the grocery store and bought a variety of fresh fruits and vegetables, lean proteins, whole grains, and healthy snacks. I also picked up some dairy products, nuts, seeds, and a few pantry staples like olive oil, spices, and herbs. I made sure to choose organic and locally sourced produce whenever possible to support local farmers and reduce my environmental impact. I also picked up

"I went to the grocery store and bought a variety of fresh fruits and vegetables, lean proteins, whole grains, and healthy snacks. I also picked up some dairy products, nuts, seeds, and a few pantry staples like olive oil, spices, and herbs. I made sure to choose organic and locally sourced produce whenever possible to support local farmers and reduce my environmental impact. I also picked up some fresh herbs and spices to add flavor to my meals without adding extra salt or sugar. I'm excited to cook some healthy and delicious meals with my fresh ingredients!"

# Moving to Composer

In [35]:

import openai
from time import sleep
from openai._exceptions import RateLimitError, APIConnectionError
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import TerminalFormatter
from utils import load_prompt, DynamicObservation, IterableDynamicObservation
import time
from LLM_cache import DiskCache
from arguments import get_config
from LMP import *

class LMP:
    """Language Model Program (LMP), adopted from Code as Policies."""
    def __init__(self, name, cfg, fixed_vars, variable_vars, debug=False, env='rlbench', model=None):
        # fixed vars and variable vars define function interface available to LLM
        # fixed vars are static, come from transforms3d library
        # variable vars are VoxPoser defined functions, defined in interfaces.py.
        #  Includes access to other low-level LMPs, i.e. planner can call composer. 
        self._name = name # composer, planner, affordance map generator, etc
        self._cfg = cfg # cfg['lmp_config']['lmps'][name]
        self._debug = debug
        self._base_prompt = load_prompt(f"{env}/{self._cfg['prompt_fname']}.txt")
        self._stop_tokens = list(self._cfg['stop'])
        self._fixed_vars = fixed_vars
        self._variable_vars = variable_vars
        self.exec_hist = ''
        self._context = None
        self._cache = DiskCache(load_cache=self._cfg['load_cache'])
        
        
        if self._cfg['use_local']: # whether steering with local model vs openai api call
             if model is None:
                 self.steer_cfg = get_config(config_path='./configs/steering.yaml')
                 self.model = SteeredModel(self.steer_cfg)
             else:
                 print(f"using passed in model for LMP {self._name}")
                 self.model = model

    def clear_exec_hist(self):
        self.exec_hist = ''

    def build_prompt(self, query):
        if len(self._variable_vars) > 0:
            variable_vars_imports_str = f"from utils import {', '.join(self._variable_vars.keys())}"
        else:
            variable_vars_imports_str = ''
        prompt = self._base_prompt.replace('{variable_vars_imports}', variable_vars_imports_str)

        if self._cfg['maintain_session'] and self.exec_hist != '':
            prompt += f'\n{self.exec_hist}'
        
        prompt += '\n'  # separate prompted examples with the query part

        if self._cfg['include_context']:
            assert self._context is not None, 'context is None'
            prompt += f'\n{self._context}'

        user_query = f'{self._cfg["query_prefix"]}{query}{self._cfg["query_suffix"]}'
        prompt += f'\n{user_query}'

        return prompt, user_query
    
    def _cached_api_call(self, **kwargs):
        # check whether completion endpoint or chat endpoint is used
        if kwargs['model'] != 'gpt-3.5-turbo-instruct' and any([chat_model in kwargs['model'] for chat_model in ['gpt-3.5', 'gpt-4']]):
            # add special prompt for chat endpoint
            user1 = kwargs.pop('prompt')
            new_query = '# Query:' + user1.split('# Query:')[-1]
            user1 = ''.join(user1.split('# Query:')[:-1]).strip()
            user1 = f"I would like you to help me write Python code to control a robot arm operating in a tabletop environment. Please complete the code every time when I give you new query. Pay attention to appeared patterns in the given context code. Be thorough and thoughtful in your code. Do not include any import statement. Do not repeat my question. Do not provide any text explanation (comment in code is okay). I will first give you the context of the code below:\n\n```\n{user1}\n```\n\nNote that x is back to front, y is left to right, and z is bottom to up."
            assistant1 = f'Got it. I will complete what you give me next.'
            user2 = new_query
            # handle given context (this was written originally for completion endpoint)
            if user1.split('\n')[-4].startswith('objects = ['):
                obj_context = user1.split('\n')[-4]
                # remove obj_context from user1
                user1 = '\n'.join(user1.split('\n')[:-4]) + '\n' + '\n'.join(user1.split('\n')[-3:])
                # add obj_context to user2
                user2 = obj_context.strip() + '\n' + user2
            messages=[
                {"role": "system", "content": "You are a helpful assistant that pays attention to the user's instructions and writes good python code for operating a robot arm in a tabletop environment."},
                {"role": "user", "content": user1},
                {"role": "assistant", "content": assistant1},
                {"role": "user", "content": user2},
            ]
            kwargs['messages'] = messages
            # print kwargs to a file
            
            if kwargs in self._cache:
                print('(using cache)', end=' ')
                return self._cache[kwargs], kwargs
            else:
                response = openai.chat.completions.create(**kwargs)
                ret = response.choices[0].message.content
                # post processing
                ret = ret.replace('```', '').replace('python', '').strip()
                self._cache[kwargs] = ret
                return ret, kwargs
        else:
            if kwargs in self._cache:
                print('(using cache)', end=' ')
                return self._cache[kwargs], kwargs
            else:
                response = openai.completions.create(**kwargs)['choices'][0]['text'].strip()
                ret = response.choices[0].text.strip()
                self._cache[kwargs] = ret
                return ret, kwargs
    
    def _chat_template(self, messages):
        prompt = ''
        for message in messages:
            if message['role'] == 'user':
                prompt += f'{message["content"]}\n'
            elif message['role'] == 'assistant':
                prompt += f'{message["content"]}\n'
        return prompt
    
    def _local_call(self, **kwargs):
        user1 = kwargs.pop('prompt')
        new_query = '# Query:' + user1.split('# Query:')[-1]
        user1 = ''.join(user1.split('# Query:')[:-1]).strip()
        user1 = f"I would like you to help me write Python code to control a robot arm operating in a tabletop environment. Please complete the code every time when I give you new query. Pay attention to appeared patterns in the given context code. Be thorough and thoughtful in your code. Do not include any import statement. Do not repeat my question. Do not provide any text explanation (comment in code is okay). I will first give you the context of the code below:\n\n```\n{user1}\n```\n\nNote that x is back to front, y is left to right, and z is bottom to up. Your task is just to write clean and concise code that follows the template above based on the objects and query supplied below."
        user2 = new_query
        
        if user1.split('\n')[-4].startswith('objects = ['):
            obj_context = user1.split('\n')[-4]
            user1 = '\n'.join(user1.split('\n')[:-4]) + '\n' + '\n'.join(user1.split('\n')[-3:])
            user2 = obj_context.strip() + '\n' + user2
        messages=[
            {"role": "system", "content": "You are a helpful assistant that pays attention to the user's instructions and writes good python code for operating a robot arm in a tabletop environment."},
            {"role": "user", "content": user1},
            {"role": "user", "content": user2},
        ]
        kwargs['messages'] = messages

        kwargs['local'] = True

        if kwargs in self._cache:
            print("using cached response yeehaw")
            return self._cache[kwargs], kwargs
        
        prompt = self._chat_template(messages)

        # print("prompting model with", prompt)
        
        ret = self.model.generate(prompt)

        self._cache[kwargs] = ret
        
        return ret, kwargs

    

    def __call__(self, query, **kwargs):
        prompt, user_query = self.build_prompt(query)

        start_time = time.time()
        while True:
            try:
                if self._cfg['use_local']:
                    code_str, _ = self._local_call(
                        prompt=prompt,
                        stop=self._stop_tokens,
                        temperature=self._cfg['temperature'],
                        max_tokens=self._cfg['max_tokens']
                    )
                    print(f'*** Local call took {time.time() - start_time:.2f}s ***')
                else:
                    code_str, _ = self._cached_api_call(
                        prompt=prompt,
                        stop=self._stop_tokens,
                        temperature=self._cfg['temperature'],
                        model=self._cfg['model'],
                        max_tokens=self._cfg['max_tokens']
                    )
                    print(f'*** OpenAI API call took {time.time() - start_time:.2f}s ***')
                break
            except (RateLimitError, APIConnectionError) as e:
                print(f'OpenAI API got err {e}')
                print('Retrying after 3s.')
                sleep(3)

        if self._cfg['include_context']:
            assert self._context is not None, 'context is None'
            to_exec = f'{self._context}\n{code_str}'
            to_log = f'{self._context}\n{user_query}\n{code_str}'
        else:
            to_exec = code_str
            to_log = f'{user_query}\n{to_exec}'

        to_log_pretty = highlight(to_log, PythonLexer(), TerminalFormatter())

        if self._cfg['include_context']:
            print('#'*40 + f'\n## "{self._name}" generated code\n' + f'## context: "{self._context}"\n' + '#'*40 + f'\n{to_log_pretty}\n')
        else:
            print('#'*40 + f'\n## "{self._name}" generated code\n' + '#'*40 + f'\n{to_log_pretty}\n')
        
        gvars = merge_dicts([self._fixed_vars, self._variable_vars])
        lvars = kwargs
        if self._debug:
            return(to_exec, prompt, user_query)

        # return function instead of executing it so we can replan using latest obs（do not do this for high-level UIs)
        if not self._name in ['composer', 'planner']:
            to_exec = 'def ret_val():\n' + to_exec.replace('ret_val = ', 'return ')
            to_exec = to_exec.replace('\n', '\n    ')

        if self._debug:
            # only "execute" function performs actions in environment, so we comment it out
            action_str = ['execute(']
            try:
                for s in action_str:
                    exec_safe(to_exec.replace(s, f'# {s}'), gvars, lvars)
            except Exception as e:
                print(f'Error: {e}')
                import pdb ; pdb.set_trace()
        else:
            exec_safe(to_exec, gvars, lvars)

        self.exec_hist += f'\n{to_log.strip()}'

        if self._cfg['maintain_session']:
            self._variable_vars.update(lvars)

        if self._cfg['has_return']:
            if self._name == 'parse_query_obj':
                try:
                    # there may be multiple objects returned, but we also want them to be unevaluated functions so that we can access latest obs
                    return IterableDynamicObservation(lvars[self._cfg['return_val_name']])
                except AssertionError:
                    return DynamicObservation(lvars[self._cfg['return_val_name']])
            return lvars[self._cfg['return_val_name']]


In [39]:
variable_vars

{'cm2index': <bound method LMP_interface.cm2index of <interfaces.LMP_interface object at 0x7af0e161c560>>,
 'detect': <bound method LMP_interface.detect of <interfaces.LMP_interface object at 0x7af0e161c560>>,
 'execute': <bound method LMP_interface.execute of <interfaces.LMP_interface object at 0x7af0e161c560>>,
 'get_ee_pos': <bound method LMP_interface.get_ee_pos of <interfaces.LMP_interface object at 0x7af0e161c560>>,
 'get_empty_affordance_map': <bound method LMP_interface.get_empty_affordance_map of <interfaces.LMP_interface object at 0x7af0e161c560>>,
 'get_empty_avoidance_map': <bound method LMP_interface.get_empty_avoidance_map of <interfaces.LMP_interface object at 0x7af0e161c560>>,
 'get_empty_gripper_map': <bound method LMP_interface.get_empty_gripper_map of <interfaces.LMP_interface object at 0x7af0e161c560>>,
 'get_empty_rotation_map': <bound method LMP_interface.get_empty_rotation_map of <interfaces.LMP_interface object at 0x7af0e161c560>>,
 'get_empty_velocity_map': <bo

In [37]:
composer = LMP(
  'composer', lmps_config['composer'], fixed_vars, variable_vars, True, env_name, model
)

using passed in model for LMP composer


In [25]:
to_exec, prompt, user_query = composer("grab my favorite snack")

using cached response yeehaw
*** Local call took 0.00s ***
########################################
## "composer" generated code
########################################
# Query: grab my favorite snack.
movable = parse_query_obj('gripper')
affordance_map = get_affordance_map('a point at the center of my favorite snack')
gripper_map = get_gripper_map('open everywhere except 1cm around my favorite snack')
execute(movable, affordance_map=affordance_map, gripper_map=gripper_map)




In [30]:
composer.model

In [29]:
print(to_exec)

movable = parse_query_obj('gripper')
affordance_map = get_affordance_map('a point at the center of my favorite snack')
gripper_map = get_gripper_map('open everywhere except 1cm around my favorite snack')
execute(movable, affordance_map=affordance_map, gripper_map=gripper_map)


In [26]:
with open("/workspace/SteerKep/steer-data/junk-healthy.json") as f:
    dset = json.load(f)

UnicodeDecodeError: 'ascii' codec can't decode byte 0xc3 in position 4426: ordinal not in range(128)

The X11 connection broke (error 1). Did the X11 server die?
QMutex: destroying locked mutex


# Training New Steering Vectors

In [23]:


behavior_dataset = SteeringDataset(
    tokenizer=tokenizer,
    examples=[(item["input"], item["input"]) for item in dset],
    suffixes=list(zip([item["compliant_continuation"] for item in dset], [item["non_compliant_continuation"] for item in dset]))
)

UnicodeDecodeError: 'ascii' codec can't decode byte 0xc3 in position 4426: ordinal not in range(128)

In [5]:
steer_vector = SteeringVector.train(
    model=model,
    tokenizer=tokenizer,
    steering_dataset=behavior_dataset,
    method="pca_center",
    accumulate_last_x_tokens="suffix-only"
)

Training steering vector

Reading representations for 729 inputs

... accumulating suffix-only hidden states

Output()

Output()

In [14]:
from activation_steering import MalleableModel
malleable_model = MalleableModel(model=model, tokenizer=tokenizer)

malleable_model.steer(
    behavior_vector=steer_vector,
    behavior_layer_ids= [23, 24, 25, 26, 27, 28, 29, 30, 31],
    behavior_vector_strength=1.4,
)

... The target model type is llama.

Steering...

In [ ]:
def load_base_prompt(lmp):
    base_dir = "/workspace/SteerKep/SteerPoser/src/prompts"

In [ ]:
def generate_openai(prompts):
    

In [16]:
def generate_local(prompts, steer):
    if(steer):
        steered_responses = malleable_model.respond_batch_sequential(
            prompts=test_prompts
        )
        return(steered_responses)
    else:
        inputs = self.tokenizer(prompts, return_tensors="pt", padding=True).to(self.model.device)
        outputs = self.model.generate(**inputs, **self.settings)
        responses = self.tokenizer.batch_decode(outputs, skip_special_tokens=True)
        return(responses)

In [13]:
# test if the model works with this new steering vector

# test_prompts = [
#   "She reached into the drawer for something to eat her lunch with and picked out a ",
#   "At the daycare, utensils were handed out for snack time. Most children were given a ",
#   "During turbulence, the passenger tried to eat their meal as safely as possible and they used ",
#   "The kitchen staff replaced all utensils after the incident involving a fork, they only purchased "
# ]

test_prompts = [
  "For her lunch break, she walked to the nearest food court",
  "After dinner, they wanted something sweet",
  "She placed an online order for groceries late at night",
  "During the hike, he took out something from his backpack to eat",
  "At the buffet, she filled her plate with her favorite items"
]

... The target model type is llama.

Steering...

['and grabbed a delicious salad from one of the many options available. The vibrant colors and fresh ingredients caught her eye as she made her selection. She added some grilled chicken for protein and a light vinaquenzo dressing on the side.\n\nAs she', 'After a delicious and filling dinner, your friends or family might be craving something sweet to satisfy their taste buds. Here are some ideas for a simple and satisfying dessert options that you can prepare:\n\n1. Fresh fruit salad: A refreshing mix of seasonal fruits', 'and received a confirmation email shortly after. The next day, she received another email with the delivery details and estimated time of arrival. She was excited to have her groceries delivered right to her door, especially since it was raining outside.\n\nWhen the delivery truck', "almonds, a granola bar, or an apple. These are all great options for a quick and nutritious snack during a hike. If you're looking for more ideas, here are some other suggestions:\n\n1. 

In [11]:
steer_vector.directions[0].shape

(4096,)

In [ ]:
steer_vector.save('/workspace/SteerKep/SteerPoser/src/svec/junk-healthy')